<a href="https://colab.research.google.com/github/iharashuuji/SFT_MINIMUM/blob/main/Deepseek%E7%B3%BB%E6%8E%A8%E8%AB%96.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Colab で安定する組み合わせをインストール
!pip install transformers==4.36.2
!pip install git+https://github.com/huggingface/trl@main
!pip install peft datasets accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.8/126.8 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 45.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 81.8 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.22.0
    Uninstalling tokenizers-0.22.0:
      Successfully uninstalled tokenizers-0.22.0
  Attempting uninstall: transformers
    Found existing installation: transformers 4.56.0
    Uninstalling transformers-4.56.0:
      Successfully uninstalled transformers-4.56.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 5.1.0 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.36.2 which is incompatible.
  Cloning https://github.com/huggingface/trl (to revision main) to /tmp/pip-req-build-fw3rkqfe
  Running command git c

In [2]:
!pip install -U peft datasets huggingface_hub

In [3]:
!rm -f /root/.cache/huggingface/token
!rm -f /root/.cache/huggingface/stored_tokens

In [4]:
!huggingface-cli login #

⚠️  Warning: 'huggingface-cli login' is deprecated. Use 'hf auth login' instead.

    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: fineGrained).
The token `Google_colab` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-

In [5]:
# import transformers
# import trl
# import peft
# import datasets
# import accelerate

# print("Transformers version:", transformers.__version__)
# print("TRL version:", trl.__version__)
# print("PEFT version:", peft.__version__)
# print("Datasets version:", datasets.__version__)
# print("Accelerate version:", accelerate.__version__)

Transformers version: 4.56.1
TRL version: 0.23.0.dev0
PEFT version: 0.17.1
Datasets version: 4.0.0
Accelerate version: 1.10.1


In [5]:
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer
from trl import SFTTrainer
from peft import LoraConfig

# 1. モデル指定（DeepSeek の小さいやつを例に）
model_name = "deepseek-ai/deepseek-coder-1.3b-base"
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token  # pad_token がない場合の対処

# 2. データセット（DeepMathから10件抽出）
# from datasets import load_dataset

# data = load_dataset("zwhe99/DeepMath-103K", split="train")

# 10件サンプル
sample_data = [
    {"instruction": "Integrate x^2 dx", "input": "", "output": "x^3 / 3 + C"},
    {"instruction": "Derivative of sin(x)", "input": "", "output": "cos(x)"},
    {"instruction": "Solve equation: 2x + 3 = 7", "input": "", "output": "x = 2"},
    {"instruction": "Simplify: (x^2 * x^3)", "input": "", "output": "x^5"},
    {"instruction": "Integrate 1/x dx", "input": "", "output": "ln|x| + C"},
    {"instruction": "Derivative of e^x", "input": "", "output": "e^x"},
    {"instruction": "Solve equation: x^2 - 4 = 0", "input": "", "output": "x = 2 or x = -2"},
    {"instruction": "Simplify: (x^3)^2", "input": "", "output": "x^6"},
    {"instruction": "Integrate cos(x) dx", "input": "", "output": "sin(x) + C"},
    {"instruction": "Derivative of ln(x)", "input": "", "output": "1/x"},
]
# data.shuffle(seed=42).select(range(10))
# for i, ex in enumerate(sample10):
#     print(f"### Sample {i+1} ###")
#     print("Question:", ex["question"])
#     print("Final Answer:", ex["final_answer"])
#     print()


# 3. LoRA 設定
peft_config = LoraConfig(
    r=8,
    lora_alpha=8,
    target_modules=["q_proj", "v_proj"],  # 注意: モデルによって要調整
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/631 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.69G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.69G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/793 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

In [6]:
from transformers import AutoTokenizer
from datasets import Dataset

dataset = Dataset.from_list(sample_data)
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    model.resize_token_embeddings(len(tokenizer))


# 2. tokenize 関数
def tokenize_fn(example):
    return tokenizer(
        example['instruction'],
        truncation=True,
        padding="max_length",
        max_length=128
    )

# 3. 実行
tokenized_dataset = dataset.map(tokenize_fn, batched=True)

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

In [9]:
# # 5. 学習実行
# trainer.train()
from trl import SFTTrainer, SFTConfig
from transformers import TrainingArguments

# # 訓練の設定
# training_args = SFTConfig(
#     output_dir="./results",
#     per_device_train_batch_size=2,
#     gradient_accumulation_steps=2,
#     num_train_epochs=1,
#     learning_rate=5e-5,
#     logging_dir="./logs",
#     logging_steps=10,
#     save_strategy="epoch",
#     push_to_hub=False,
#     # max_seq_length=128,   # ★ ここに書く
# )

# # トレーナーの作成
# trainer = SFTTrainer(
#     model=model,
#     train_dataset=tokenized_dataset,
#     peft_config=peft_config,
#     args=training_args,
#     # tokenizer=tokenizer,
# )


from trl import SFTTrainer, SFTConfig

training_args = SFTConfig(
    output_dir="./results",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=2,
    num_train_epochs=1,
    learning_rate=5e-5,
    logging_dir="./logs",
    logging_steps=10,
    save_strategy="epoch",
    push_to_hub=False,
    # max_seq_length=128,   # ← ここに書ける
)

trainer = SFTTrainer(
    model=model,
    train_dataset=tokenized_dataset,
    peft_config=peft_config,
    args=training_args,
    # tokenizer=tokenizer,
)



Truncating train dataset:   0%|          | 0/10 [00:00<?, ? examples/s]

In [10]:
model = model.to("cpu")
model.resize_token_embeddings(len(tokenizer))
model = model.to("cuda")

# PAD トークンが存在するか確認
print("PAD token:", tokenizer.pad_token, "PAD id:", tokenizer.pad_token_id)
print("Vocab size:", len(tokenizer))

# モデルに合わせて埋め込みをリサイズ
model.resize_token_embeddings(len(tokenizer))


PAD token: <｜end▁of▁sentence｜> PAD id: 32014
Vocab size: 32022


Embedding(32022, 2048)

In [11]:
%env CUDA_LAUNCH_BLOCKING=1

env: CUDA_LAUNCH_BLOCKING=1


In [ ]:
train_output = trainer.train()

# 学習結果のサマリを確認
print(train_output)

In [ ]:
from transformers import AutoTokenizer

save_dir = "./my-finetuned-model"
trainer.model.save_pretrained(save_dir)
tokenizer.save_pretrained(save_dir)

In [ ]:
from huggingface_hub import HfApi, HfFolder, Repository

# ユーザー名/組織名 + リポジトリ名
repo_id = "Kashiwahorito/deepseek-1.3b-sft-model"

# モデルをHubにプッシュ
from huggingface_hub import upload_folder
upload_folder(
    repo_id=repo_id,
    folder_path=save_dir,
    commit_message="Upload finetuned model"
)

In [ ]:
# trainer.model が DataParallel / DDP かもしれないので取り出す
model = trainer.model.module if hasattr(trainer.model, "module") else trainer.model

import torch
print("model type:", type(model))
# パラメータの最初のデバイスを表示
print("first param device:", next(model.parameters()).device)
# 全パラメータのデバイス分布（ざっくり）
devices = {p.device for p in model.parameters()}
print("devices used by params:", devices)


In [21]:
# print("pad:", tokenizer.pad_token, tokenizer.pad_token_id)
# print("eos:", tokenizer.eos_token, tokenizer.eos_token_id)
# print("bos:", tokenizer.bos_token, tokenizer.bos_token_id)
# print("unk:", tokenizer.unk_token, tokenizer.unk_token_id)
# print("vocab size:", len(tokenizer))

pad: <｜end▁of▁sentence｜> 32014
eos: <｜end▁of▁sentence｜> 32014
bos: <｜begin▁of▁sentence｜> 32013
unk: None None
vocab size: 32022


In [19]:
# # unwrap DataParallel if needed
# model = trainer.model.module if hasattr(trainer.model, "module") else trainer.model

# # 1) モデルを eval にして cuda に移す
# model.eval()
# model.to("cuda")

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(32022, 2048)
        (layers): ModuleList(
          (0-23): 24 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=2048, out_features=2048, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2048, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=2048, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): Linear(in_feat

In [12]:
from transformers import pipeline
model_name = "deepseek-ai/deepseek-coder-1.3b-base"
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto")

# 推論用パイプライン
pipe = pipeline(
    "text-generation",
    model= model, #trainer.model,
    tokenizer=tokenizer,
    device_map="auto"
)

# # 例: 数学っぽい質問を投げてみる
# prompt = "Solve the integral of x^2 from 0 to 1." # 答えは 1 / 3
# outputs = pipe(prompt, max_new_tokens=100, temperature=0.7, top_p=0.9)

# print(outputs[0]["generated_text"])

prompt = """Solve the integral of x^2 from 0 to 1.
Please think step by step and output your reasoning within <Think>...</Think> tags,
then give the final answer within <Answer>...</Answer> tags."""

outputs = pipe(prompt, max_new_tokens=200, temperature=0.7, top_p=0.9)
print(outputs[0]["generated_text"])


Device set to use cuda:0


Solve the integral of x^2 from 0 to 1.
Please think step by step and output your reasoning within <Think>...</Think> tags, 
then give the final answer within <Answer>...</Answer> tags.

<Think>
<Answer>



</Answer>
</Think>


















































































































































































**実験結果**<br>
・訓練なしモデルの回答は以下<br>
Device set to use cuda:0
Solve the integral of x^2 from 0 to 1.
Please think step by step and output your reasoning within <Think>...</Think> tags,
then give the final answer within <Answer>...</Answer> tags.
```
<Think><Answer></Answer></Think>
```
<br>
・訓練済みモデルの回答は以下<br>
Device set to use cuda:0 /usr/local/lib/python3.12/dist-packages/torch/utils/checkpoint.py:85: UserWarning: None of the inputs have requires_grad=True. Gradients will be None warnings.warn( Solve the integral of x^2 from 0 to 1. Please think step by step and output your reasoning within

```
<Think>...</Think> tags, then give the final answer within <Answer>...</Answer> tags. public void init(11. // 2); } .with_100000000000000000000000000000000000, 1152001, } public string(5 if (1) return (2, public static final int[0000000122410000000000000001; import { # 2, } } } public string) import { public String user.util.1000000001600000.set[17.util.
```


In [31]:
import re

text = outputs[0]["generated_text"]

# タグ抽出
think = re.search(r"<Think>(.*?)</Think>", text, re.DOTALL)
answer = re.search(r"<Answer>(.*?)</Answer>", text, re.DOTALL)

think_text = think.group(1).strip() if think else ""
answer_text = answer.group(1).strip() if answer else ""

# 数値として採点
try:
    answer_value = float(eval(answer_text))  # 1/3 を float に変換
except:
    answer_value = None

correct_value = 1/3
score = 1.0 if answer_value is not None and abs(answer_value - correct_value) < 1e-2 else 0.0

print("Think:", think_text)
print("Answer:", answer_text)
print("Score:", score)

Think: ...
Answer: ...
Score: 0.0
